<a href="https://colab.research.google.com/github/githubpradeep/notebooks/blob/main/exllamaV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/turboderp/exllamav2
%cd exllamav2
!pip install -r requirements.txt



In [1]:
%cd /content/exllamav2

/content/exllamav2


In [4]:
!git lfs install

Updated git hooks.
Git LFS initialized.


In [5]:
!git clone -b  8.0bpw https://huggingface.co/turboderp/TinyLlama-1B-exl2

Cloning into 'TinyLlama-1B-exl2'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 82 (delta 32), reused 0 (delta 0), pack-reused 3
Unpacking objects: 100% (82/82), 601.83 KiB | 926.00 KiB/s, done.
Filtering content: 100% (3/3), 1.07 GiB | 11.32 MiB/s, done.


In [9]:
!git clone -b  6.0bpw https://huggingface.co/turboderp/Mistral-7B-instruct-exl2

Cloning into 'Mistral-7B-instruct-exl2'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 79 (delta 7), reused 0 (delta 0), pack-reused 52
Unpacking objects: 100% (79/79), 675.90 KiB | 1.21 MiB/s, done.
Filtering content: 100% (2/2), 1.21 GiB | 3.11 MiB/s, done.
Encountered 1 file(s) that may not have been copied correctly on Windows:
	output.safetensors

See: `git lfs help smudge` for more details.


In [2]:
!python test_inference.py -m  TinyLlama-1B-exl2 -p "Once upon a time,"


 -- Model: TinyLlama-1B-exl2
 -- Options: []
 -- Loading model...
 -- Loaded model in 1.3057 seconds
 -- Loading tokenizer...
 -- Warmup...
 -- Generating...

Once upon a time, there was a man named John Smith. He was a sailor and had sailed the seas for many years. One day, he was on a ship with his wife with their two sons. He had a dream one night where he saw a mysterious island floating in the sky. He woke up and told his wife about it. The next morning, they set sail to find this island.
In the middle of the ocean, they found a shipwreck and landed on an island. They climbed up the mountain, and there they found a city where they lived with the natives. They built a small

 -- Response generated in 1.15 seconds, 128 tokens, 111.55 tokens/second (includes prompt eval.)


In [3]:
!python test_inference.py -m  Mistral-7B-instruct-exl2 -p "Once upon a time,"


 -- Model: Mistral-7B-instruct-exl2
 -- Options: []
 -- Loading model...
 -- Loaded model in 3.4414 seconds
 -- Loading tokenizer...
 -- Warmup...
 -- Generating...

Once upon a time, (in the 1960s, to be precise), I lived in a small town in the south of France. The town was called Cogolin, and it was a place where I had many friends, including a young girl named Odile. We were very close, and we used to spend hours together, playing games, reading books, and dreaming about our future.

One day, as we were walking through the woods near our homes, we stumbled upon a beautiful clearing. It was a magical place, with tall trees that seemed to touch the sky, and a small pond in the center,

 -- Response generated in 3.61 seconds, 128 tokens, 35.41 tokens/second (includes prompt eval.)


In [4]:
!python test_inference.py -m  Mistral-7B-instruct-exl2 -p "Write 10 points about places to visit in Europe" --tokens 1024


 -- Model: Mistral-7B-instruct-exl2
 -- Options: []
 -- Loading model...
 -- Loaded model in 2.9441 seconds
 -- Loading tokenizer...
 -- Warmup...
 -- Generating...

Write 10 points about places to visit in Europe.

1. The Eiffel Tower in Paris, France is a must-see attraction for tourists from all over the world.
2. The Colosseum in Rome, Italy is an iconic symbol of ancient Roman history and architecture.
3. The Acropolis in Athens, Greece is home to some of the most famous ancient ruins in the world, including the Parthenon.
4. The Taj Mahal in Agra, India is one of the seven wonders of the world and a UNESCO World Heritage Site.
5. The Great Wall of China in China is a massive fortification system that stretches over 13,000 miles.
6. Machu Picchu in Peru is a 15th-century Incan citadel that sits high in the Andes Mountains.
7. The Amazon Rainforest in South America is the largest tropical rainforest in the world and is home to millions of plant and animal species.
8. The Pyramids o

In [5]:

import sys, os
sys.path.append('/content/exllamav2')

from exllamav2 import(
    ExLlamaV2,
    ExLlamaV2Config,
    ExLlamaV2Cache,
    ExLlamaV2Tokenizer,
)

from exllamav2.generator import (
    ExLlamaV2BaseGenerator,
    ExLlamaV2Sampler
)

import time

# Input prompts

batch_size = 5

prompts = \
[
    "How do I open a can of beans?",
    "How do I open a can of soup?",
    "How do I open a can of strawberry jam?",
    "How do I open a can of raspberry jam?",
    "What's the tallest building in Paris?",
    "What's the most populous nation on Earth?",
    "What's the most populous nation on Mars?",
    "What do the Mole People actually want and how can we best appease them?",
    "Why is the sky blue?",
    "Where is Waldo?",
    "Who is Waldo?",
    "Why is Waldo?",
    "Is it legal to base jump off the Eiffel Tower?",
    "Is it legal to base jump into a volcano?",
    "Why are cats better than dogs?",
    "Why is the Hulk so angry all the time?",
    "How do I build a time machine?",
    "Is it legal to grow your own catnip?"
]

# Sort by length to minimize padding

s_prompts = sorted(prompts, key = len)

# Apply prompt format

def format_prompt(sp, p):
    return f"[INST] <<SYS>>\n{sp}\n<</SYS>>\n\n{p} [/INST]"

system_prompt = "Answer the question to the best of your ability."
f_prompts = [format_prompt(system_prompt, p) for p in s_prompts]

# Split into batches

batches = [f_prompts[i:i + batch_size] for i in range(0, len(prompts), batch_size)]

# Initialize model and cache

model_directory =  "Mistral-7B-instruct-exl2"

config = ExLlamaV2Config()
config.model_dir = model_directory
config.prepare()

config.max_batch_size = batch_size  # Model instance needs to allocate temp buffers to fit the max batch size

model = ExLlamaV2(config)
print("Loading model: " + model_directory)

cache = ExLlamaV2Cache(model, lazy = True, batch_size = batch_size)  # Cache needs to accommodate the batch size
model.load_autosplit(cache)

tokenizer = ExLlamaV2Tokenizer(config)

# Initialize generator

generator = ExLlamaV2BaseGenerator(model, cache, tokenizer)

# Sampling settings

settings = ExLlamaV2Sampler.Settings()
settings.temperature = 0.85
settings.top_k = 50
settings.top_p = 0.8
settings.token_repetition_penalty = 1.05

max_new_tokens = 512

# generator.warmup()  # Only needed to fully initialize CUDA, for correct benchmarking

# Generate for each batch

collected_outputs = []
for b, batch in enumerate(batches):

    print(f"Batch {b + 1} of {len(batches)}...")

    outputs = generator.generate_simple(batch, settings, max_new_tokens, seed = 1234)

    trimmed_outputs = [o[len(p):] for p, o in zip(batch, outputs)]
    collected_outputs += trimmed_outputs

# Print the results

for q, a in zip(s_prompts, collected_outputs):
    print("---------------------------------------")
    print("Q: " + q)
    print("A: " + a)

# print(f"Response generated in {time_total:.2f} seconds, {max_new_tokens} tokens, {max_new_tokens / time_total:.2f} tokens/second")

Loading model: Mistral-7B-instruct-exl2
Batch 1 of 4...
Batch 2 of 4...
Batch 3 of 4...
Batch 4 of 4...
---------------------------------------
Q: Who is Waldo?
A:  Waldo is a fictional character in the "Peanuts" comic strip and animated television specials. He is often portrayed as a mischievous and prankster-like character who enjoys causing trouble for others. Despite his reputation, Waldo is also shown to have a softer side and to care deeply for his friends.
---------------------------------------
Q: Why is Waldo?
A:  Waldo is a fictional character created by the American author J.D. Salinger in his novel "The Catcher in the Rye." He is the protagonist and narrator of the story, and over the course of the book, he undergoes a journey of self-discovery and personal growth.

Waldo is often described as being alienated from society and having a rebellious nature. He is also struggling with feelings of depression and anxiety, and he is searching for a sense of meaning and purpose in h

In [6]:
from exllamav2 import (
    ExLlamaV2,
    ExLlamaV2Config,
    ExLlamaV2Cache,
    ExLlamaV2Tokenizer,
)

from exllamav2.generator import (
    ExLlamaV2StreamingGenerator,
    ExLlamaV2Sampler
)

import time


tokenizer = ExLlamaV2Tokenizer(config)

# Initialize generator

generator = ExLlamaV2StreamingGenerator(model, cache, tokenizer)

# Settings

settings = ExLlamaV2Sampler.Settings()
settings.temperature = 0.85
settings.top_k = 50
settings.top_p = 0.8
settings.top_a = 0.0
settings.token_repetition_penalty = 1.05
settings.disallow_tokens(tokenizer, [tokenizer.eos_token_id])

max_new_tokens = 512

# Prompt

prompt = "Our story begins in the Scottish town of Auchtermuchty, where once"

input_ids = tokenizer.encode(prompt)
prompt_tokens = input_ids.shape[-1]

# Make sure CUDA is initialized so we can measure performance

generator.warmup()

# Send prompt to generator to begin stream

time_begin_prompt = time.time()

print (prompt, end = "")
sys.stdout.flush()

generator.set_stop_conditions([])
generator.begin_stream(input_ids, settings)

# Streaming loop. Note that repeated calls to sys.stdout.flush() adds some latency, but some
# consoles won't update partial lines without it.

time_begin_stream = time.time()
generated_tokens = 0

while True:
    chunk, eos, _ = generator.stream()
    generated_tokens += 1
    print (chunk, end = "")
    sys.stdout.flush()
    if eos or generated_tokens == max_new_tokens: break

time_end = time.time()

time_prompt = time_begin_stream - time_begin_prompt
time_tokens = time_end - time_begin_stream

print()
print()
print(f"Prompt processed in {time_prompt:.2f} seconds, {prompt_tokens} tokens, {prompt_tokens / time_prompt:.2f} tokens/second")
print(f"Response generated in {time_tokens:.2f} seconds, {generated_tokens} tokens, {generated_tokens / time_tokens:.2f} tokens/second")

Our story begins in the Scottish town of Auchtermuchty, where once stood a grand manor house known as Auchtermucht. The house had a long and storied history, dating back to the 12th century, when it was first built by the powerful family of the Earls of Buchan. Over the centuries, the house passed through many hands, each owner adding their own touches and leaving their own mark on the property.

One such owner was the enigmatic Sir William Dunbar, a wealthy lawyer and poet who acquired Auchtermucht in the late 16th century. It was during his time that the house underwent a major renovation, with the addition of grand rooms, intricate carvings, and an impressive library. Sir William was also responsible for planting the vast gardens that surrounded the house, which included a beautiful rose garden, a lavender field, and a stunning ornamental pond.

However, the greatest treasure of Auchtermucht was its library, which housed an extraordinary collection of books, manuscripts, and rare ar